In [1]:
%cd /kaggle/working

/kaggle/working


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!git clone https://github.com/WongKinYiu/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 710, done.
remote: Counting objects: 100% (336/336), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 710 (delta 249), reused 202 (delta 201), pack-reused 374
Receiving objects: 100% (710/710), 3.23 MiB | 18.10 MiB/s, done.
Resolving deltas: 100% (294/294), done.


In [4]:
%cd /kaggle/working/yolov9

/kaggle/working/yolov9


In [5]:
!conda install -y gdown

Retrieving notices: ...working... done
- \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [6]:
%cd /kaggle/working/
# dataset link for bangla sign language zip
# !gdown --id 1JQhewCk77RXOLryMsmDdc11q4YubHAhF
# dataset for testing
# !gdown 1-4MU6v0g_I-Lm8l4wrfeNFO9xEoVNYwO
!gdown 1-Esam-6_haIP1nps3PPKq0icvEkX3QiV

/kaggle/working
Downloading...
From (original): https://drive.google.com/uc?id=1-Esam-6_haIP1nps3PPKq0icvEkX3QiV
From (redirected): https://drive.google.com/uc?id=1-Esam-6_haIP1nps3PPKq0icvEkX3QiV&confirm=t&uuid=ebd21407-e0cc-4345-b9d1-da016f021789
To: /kaggle/working/bdsl_final.zip
100%|██████████████████████████████████████| 5.85G/5.85G [02:06<00:00, 46.1MB/s]


In [7]:
import zipfile
import os



# Step 2: Define the path to the zip file
zip_file_path = '/kaggle/working/bdsl_final.zip'

# Step 3: Extract the name of the zip file (without extension) to create a folder
zip_file_name = os.path.basename(zip_file_path)
folder_name = os.path.splitext(zip_file_name)[0]
extract_to_path = os.path.join('/kaggle/working/yolov9', folder_name)

# Step 4: Create the output directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Step 5: Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

# Verify the extraction
if os.path.exists(extract_to_path):
    print(f"Folder successfully unzipped to {extract_to_path}")
else:
    print("Failed to unzip the folder.")

Folder successfully unzipped to /kaggle/working/yolov9/bdsl_final


In [8]:
import os

file_path = '/kaggle/working/bdsl_final.zip'

# Check if the file exists before trying to delete it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted")
else:
    print(f"{file_path} does not exist")


/kaggle/working/bdsl_final.zip has been deleted


In [9]:
# %cd /kaggle/working/yolov9/models/detect
# !gdown --id 1SHNpPjhxI11Cu3Lz8ZN5lhCDLe4okBCe

In [10]:
%cd /kaggle/working/yolov9
# data.yml file
# !gdown 1yRa2jbtUw74fhIr9o73hECeSPy86z52t
# yml file for shoreo shoreya
!gdown 1LWWlVuwK6kQl9DVOAUK6kFVxhGKRFgMg

/kaggle/working/yolov9
Downloading...
From: https://drive.google.com/uc?id=1LWWlVuwK6kQl9DVOAUK6kFVxhGKRFgMg
To: /kaggle/working/yolov9/data.yaml
100%|██████████████████████████████████████████| 527/527 [00:00<00:00, 2.98MB/s]


In [11]:
%cd /kaggle/working/yolov9

/kaggle/working/yolov9


In [12]:
#!python train_dual.py --workers 16 --batch 8 --img 640 --epochs 20 --data /kaggle/working/yolov9/data.yaml --weights /kaggle/working/yolov9/yolov9-e.pt --device 0,1 --cfg /kaggle/working/yolov9/models/detect/yolov9_custom.yaml --hyp /kaggle/working/yolov9/data/hyps/hyp.scratch-high.yaml

In [13]:
# !python train_dual.py \
# --workers 16 --batch 8 --img 512 --epochs 10 --verbose True \
# --resume true --data /kaggle/working/yolov9/data.yaml \
# --weights /kaggle/working/yolov9/yolov9-e.pt --device 0,1 \
# --cfg /kaggle/working/yolov9/models/detect/yolov9_custom.yaml \
# --hyp /kaggle/working/yolov9/data/hyps/hyp.scratch-high.yaml

### 

In [14]:
!pip install ultralytics 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 570.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 5.3 MB/s eta 0:00:00


In [15]:
%cd /kaggle/working/yolov9

/kaggle/working/yolov9


In [16]:
#best pt file 
# !gdown 1Pgf9D29jDo2RrY3EbjUWsBco0lbE0k4F

In [17]:
from ultralytics import YOLO


# Build a YOLOv9c model from pretrained weight
model = YOLO("yolov9c.pt")

# Display model information (optional)
model.info()



100%|██████████| 49.4M/49.4M [00:00<00:00, 203MB/s]


YOLOv9c summary: 618 layers, 25590912 parameters, 0 gradients, 104.0 GFLOPs


(618, 25590912, 0, 104.02268160000003)

In [18]:
import torch
import torch.nn as nn


class DataParallelModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.block1 = nn.Linear(10, 20)

        # wrap block2 in DataParallel
        self.block2 = nn.Linear(20, 20)
        self.block2 = nn.DataParallel(self.block2)

        self.block3 = nn.Linear(20, 20)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        return x

In [19]:
# train.py
import wandb
import random  # for demo script
WANDB_API_KEY='957d38e93afcd97d0286c494f525efdb6caf7707'

wandb.login(key=WANDB_API_KEY)

epochs = 15
lr = 0.01

run = wandb.init(
    # Set the project where this run will be logged
    project="Bangla Sign Language Detection",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": lr,
        "epochs": epochs,
    },
)
# offset = random.random() / 5
# print(f"lr: {lr}")

# # simulating a training run
# for epoch in range(2, epochs):
#     acc = 1 - 2**-epoch - random.random() / epoch - offset
#     loss = 2**-epoch + random.random() / epoch + offset
#     print(f"epoch={epoch}, accuracy={acc}, loss={loss}")
#     wandb.log({"accuracy": acc, "loss": loss})

# run.log_code()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mahedi615 (own-business615). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/yolov9/wandb/run-20240602_181926-ioqpsihb
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run breezy-cloud-4
wandb: ⭐️ View project at https://wandb.ai/own-business615/Bangla%20Sign%20Language%20Detection
wandb: 🚀 View run at https://wandb.ai/own-business615/Bangla%20Sign%20Language%20Detection/runs/ioqpsih

In [20]:
# Train the model on the COCO8 example dataset for 100 epochs
results = model.train(data="/kaggle/working/yolov9/data.yaml",  epochs=15, imgsz=640, verbose=True,save=True,save_period=1,device=[0,1])

Ultralytics YOLOv8.2.27 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
                                                      CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=/kaggle/working/yolov9/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=[0, 1], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

100%|██████████| 22.2M/22.2M [00:00<00:00, 117MB/s] 
2024-06-02 18:19:46,271	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-02 18:19:48,164	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=49

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    212864  ultralytics.nn.modules.block.RepNCSPELAN4    [128, 256, 128, 64, 1]        
  3                  -1  1    164352  ultralytics.nn.modules.block.ADown           [256, 256]                    
  4                  -1  1    847616  ultralytics.nn.modules.block.RepNCSPELAN4    [256, 512, 256, 128, 1]       
  5                  -1  1    656384  ultralytics.nn.modules.block.ADown           [512, 512]                    
  6                  -1  1   2857472  ultralytics.nn.modules.block.RepNCSPELAN4    [512, 512, 512, 256, 1]       
  7                  -1  1    656384  ultralytic

wandb: Currently logged in as: mahedi615 (own-business615). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/yolov9/wandb/run-20240602_182015-wrdbbgzo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run train
wandb: ⭐️ View project at https://wandb.ai/own-business615/YOLOv8
wandb: 🚀 View run at https://wandb.ai/own-business615/YOLOv8/runs/wrdbbgzo


Overriding model.yaml nc=80 with nc=49
Transferred 931/937 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 145/145 [02:25<00:00,  1.00s/it]


                   all       2308       2307       0.99       0.98      0.989      0.885
                     অ         57         57      0.991      0.965      0.987      0.885
                     আ         55         55      0.944      0.945      0.937       0.84
                     ঊ         45         45      0.961      0.978      0.993      0.904
                     এ         47         47      0.979       0.98      0.991      0.933
                     ও         47         47          1      0.965      0.995       0.91
                     ক         37         37      0.995          1      0.995       0.94
                     খ         53         53      0.981          1      0.995      0.845
                     গ         50         50      0.996          1      0.995      0.875
                     ঘ         48         48          1      0.986      0.995      0.898
                     চ         59         59      0.997      0.983      0.995      0.846
                     

/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 2437 (\N{BENGALI LETTER A}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: Matplotlib currently does not support Bengali natively.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 2438 (\N{BENGALI LETTER AA}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 2439 (\N{BENGALI LETTER I}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: Glyph 2442 (\N{BENGALI LETTER UU}) missing from current font.
  fig.savefig(plot_fname, dpi=250)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/metrics.py:427: UserWarning: G

Speed: 0.2ms preprocess, 11.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▆█▇▇▆▆▅▅▄▃▃▂▂▁
wandb:                  lr/pg1 ▃▆█▇▇▆▆▅▅▄▃▃▂▂▁
wandb:                  lr/pg2 ▃▆█▇▇▆▆▅▅▄▃▃▂▂▁
wandb:        metrics/mAP50(B) ▁▄▅▇▇▇▇████████
wandb:     metrics/mAP50-95(B) ▁▃▄▅▆▆▆▇▇▇▇▇███
wandb:    metrics/precision(B) ▁▂▃▅▆▆▇▇▇██████
wandb:       metrics/recall(B) ▁▄▄▆▆▇▇▇███████
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss ██▇▇▆▄▄▃▃▂▂▂▂▁▁
wandb:          train/cls_loss █▆▅▄▄▂▂▂▂▁▁▁▁▁▁
wandb:          train/dfl_loss ███▇▆▅▄▄▄▃▃▂▂▁▁
wandb:            val/box_loss █▆▆▅▄▄▄▃▃▂▂▂▁▁▁
wandb:            val/cls_loss █▆▅▄▃▃▃▂▂▂▁▁▁▁▁
wandb:            val/dfl_loss █▆▅▅▄▄▄▃▃▂▂▂▂▁▁
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 5e-05
wandb:                  lr/pg1 5e-05
wandb:                  lr/pg2 5e-05
wandb:        metrics/mAP50(B) 0.988

In [21]:
%cd /kaggle/working/

/kaggle/working


In [22]:
offset = random.random() / 5
print(f"lr: {lr}")

# simulating a training run
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset
    print(f"epoch={epoch}, accuracy={acc}, loss={loss}")
    wandb.log({"accuracy": acc, "loss": loss})

run.log_code()

lr: 0.01
epoch=2, accuracy=0.20213842822483913, loss=0.6291701607204321
epoch=3, accuracy=0.6198100462640026, loss=0.46430927742787914
epoch=4, accuracy=0.6673820953323868, loss=0.42733401756370276
epoch=5, accuracy=0.7392030844792048, loss=0.29545376113548083
epoch=6, accuracy=0.7182602897858186, loss=0.33586151783756546
epoch=7, accuracy=0.7512050074353631, loss=0.21695941950496733
epoch=8, accuracy=0.7327338541753374, loss=0.25008674488942606
epoch=9, accuracy=0.8013284667658302, loss=0.2719204126348141
epoch=10, accuracy=0.7318605195912251, loss=0.25088265640466856
epoch=11, accuracy=0.748612262041392, loss=0.1975678851294944
epoch=12, accuracy=0.7700521767149797, loss=0.2440317015940091
epoch=13, accuracy=0.7783794107736102, loss=0.20532511103694912
epoch=14, accuracy=0.8238616511052856, loss=0.19995767942224418


<Artifact source-Bangla_Sign_Language_Detection-None>

In [23]:
import os

# Define the path of the file to be deleted
file_path = "/kaggle/working/yolov9/bdsl_final.zip"

# Check if the file exists
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"File {file_path} has been deleted.")
else:
    print(f"The file {file_path} does not exist.")


The file /kaggle/working/yolov9/bdsl_final.zip does not exist.


In [24]:
# !zip -r file.zip /kaggle/working/yolov9/runs

In [25]:
# !ls

In [26]:
# %cd /kaggle/working/
# from IPython.display import FileLink
# FileLink('/kaggle/working/yolov9/file.zip')

In [27]:
!zip -r file.zip /kaggle/working/yolov9/runs

  adding: kaggle/working/yolov9/runs/ (stored 0%)
  adding: kaggle/working/yolov9/runs/detect/ (stored 0%)
  adding: kaggle/working/yolov9/runs/detect/train/ (stored 0%)
  adding: kaggle/working/yolov9/runs/detect/train/train_batch0.jpg (deflated 5%)
  adding: kaggle/working/yolov9/runs/detect/train/R_curve.png (deflated 12%)
  adding: kaggle/working/yolov9/runs/detect/train/train_batch2885.jpg (deflated 18%)
  adding: kaggle/working/yolov9/runs/detect/train/results.png (deflated 8%)
  adding: kaggle/working/yolov9/runs/detect/train/confusion_matrix.png (deflated 38%)
  adding: kaggle/working/yolov9/runs/detect/train/train_batch2.jpg (deflated 6%)
  adding: kaggle/working/yolov9/runs/detect/train/labels_correlogram.jpg (deflated 34%)
  adding: kaggle/working/yolov9/runs/detect/train/F1_curve.png (deflated 11%)
  adding: kaggle/working/yolov9/runs/detect/train/labels.jpg (deflated 34%)
  adding: kaggle/working/yolov9/runs/detect/train/val_batch0_labels.jpg (deflated 9%)
  adding: kaggle

In [28]:
# from ultralytics import YOLO

# # Load a pretrained YOLOv8n model
# model = YOLO("/kaggle/working/yolov9/runs/detect/train2/weights/best.pt")

# # Run inference on 'bus.jpg' with arguments
# model.predict("/kaggle/working/yolov9/bdsl_test/test/images", save=True, imgsz=320, conf=0.5)

In [29]:
# from IPython.display import Image

# # Define the path to the image file
# image_path = "/kaggle/working/yolov9/runs/detect/predict2/এ_243.jpg"

# # Display the image
# Image(filename=image_path)


In [30]:
# import os
# from IPython.display import display, Image
# from PIL import Image as PILImage

# # Specify the directory containing the images
# directory = '/kaggle/working/yolov9/runs/detect/predict3'

# # List all files in the directory
# files = os.listdir(directory)

# # Filter out only the image files (you can add more extensions if needed)
# image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

# # Loop through the list of image files and display each one
# for image_file in image_files:
#     image_path = os.path.join(directory, image_file)
#     display(PILImage.open(image_path))

# # Alternatively, using IPython.display.Image to directly display from the path
# for image_file in image_files:
#     image_path = os.path.join(directory, image_file)
#     display(Image(filename=image_path))
